In [88]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder

from sklearn.impute import SimpleImputer

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_seq_items', None)

In [89]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/test.csv')

In [90]:
holidays_events = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/holidays_events.csv')
oil = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/oil.csv')
stores = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/stores.csv')
trandsactions = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/transactions.csv')

In [5]:
train.set_index('id', inplace=True)
test.set_index('id', inplace=True)

In [91]:
train['year'] = pd.to_numeric(train['date'].str[:4])
train['month'] = pd.to_numeric(train['date'].str[5:7])
train['day'] = pd.to_numeric(train['date'].str[8:10])
train['week'] = pd.to_datetime(train['date']).dt.weekday

train['date_'] = (train['year']*10000) + (train['month']*100) + train['day']

In [92]:
test['year'] = pd.to_numeric(test['date'].str[:4])
test['month'] = pd.to_numeric(test['date'].str[5:7])
test['day'] = pd.to_numeric(test['date'].str[8:10])
test['week'] = pd.to_datetime(test['date']).dt.weekday

test['date_'] = (test['year']*10000) + (test['month']*100) + test['day']

In [93]:
trandsactions['year'] = pd.to_numeric(trandsactions['date'].str[:4])
trandsactions['month'] = pd.to_numeric(trandsactions['date'].str[5:7])
trandsactions['day'] = pd.to_numeric(trandsactions['date'].str[8:10])

trandsactions['date_'] = (trandsactions['year']*10000) + (trandsactions['month']*100) + trandsactions['day']

In [94]:
group_flag = 'store_nbr'

In [95]:
gb = train.groupby(by=group_flag)
gb2 = trandsactions.groupby(by=group_flag)
col = trandsactions[group_flag].unique()
dump = pd.DataFrame()


for i in col:
  all_data_ = gb.get_group(i).copy()
  trandsactions_ = gb2.get_group(i).copy()

  for idx, row in trandsactions_.iterrows():
    all_data_.loc[all_data_['date_'] == row['date_'], 'transactions'] = row['transactions']

  dump = pd.concat([dump, all_data_], sort=False)
    
train = dump

In [13]:
na_check = train.isna().sum() / len(train)
na_check

date            0.000000
store_nbr       0.000000
family          0.000000
sales           0.000000
onpromotion     0.000000
year            0.000000
month           0.000000
day             0.000000
date_           0.000000
week            0.000000
transactions    0.081904
dtype: float64

In [96]:
all_data = pd.concat([train, test], sort=False)

In [ ]:
na_check = all_data.isna().sum() / len(all_data)
na_check

id              0.000000
date            0.000000
store_nbr       0.000000
family          0.000000
sales           0.009412
onpromotion     0.000000
year            0.000000
month           0.000000
day             0.000000
date_           0.000000
transactions    0.059488
family_value    0.000000
isHoliday       0.000000
oil_price       0.000000
dtype: float64

In [ ]:
all_data.dtypes

id                int64
date             object
store_nbr         int64
family           object
sales           float64
onpromotion     float64
year              int64
month             int64
day               int64
date_             int64
transactions    float64
dtype: object

In [97]:
group_flag = 'year'

col = all_data['year'].unique()
col2 = all_data['family'].unique()

gb = all_data.groupby(by=group_flag)

all_data['family_value'] = 0

for i in col:
  all_data_ = gb.get_group(i).copy()


  for j in col2:
    sum = all_data_.loc[(all_data_['family'] == j) & (all_data_['sales'] > 0), 'sales'].sum()
    cnt = all_data_.loc[(all_data_['family'] == j) & (all_data_['sales'] > 0), 'sales'].count()

    all_data.loc[(all_data['family'] == j) & (all_data['year'] == i), 'family_value'] = sum / cnt


  sum = all_data_.loc[all_data_['sales'] > 0, 'sales'].sum()
  cnt = all_data_.loc[all_data_['sales'] > 0, 'sales'].count()

  all_data.loc[(all_data['family_value'].isna()) & (all_data['year'] == i), 'family_value'] = sum / cnt


all_data.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


,id,date,store_nbr,family,sales,onpromotion,year,month,day,week,date_,transactions,family_value
561,561,2013-01-01,25,AUTOMOTIVE,0.0,0.0,2013,1,1,1,20130101,770.0,5.947114
562,562,2013-01-01,25,BABY CARE,0.0,0.0,2013,1,1,1,20130101,770.0,436.931976
563,563,2013-01-01,25,BEAUTY,2.0,0.0,2013,1,1,1,20130101,770.0,3.660048
564,564,2013-01-01,25,BEVERAGES,810.0,0.0,2013,1,1,1,20130101,770.0,1287.717597
565,565,2013-01-01,25,BOOKS,0.0,0.0,2013,1,1,1,20130101,770.0,436.931976


In [98]:
col = all_data['family'].unique()

for i in col:
  all_data['family_'+i] = False
  all_data.loc[all_data['family'] == i, 'family_'+i] = True

In [99]:
group_flag = 'year'

col = all_data['year'].unique()
col2 = all_data['store_nbr'].unique()

gb = all_data.groupby(by=group_flag)


for i in col:
  all_data_ = gb.get_group(i).copy()

  for j in col2:
    sum = all_data_.loc[all_data_['store_nbr'] == j, 'transactions'].sum()
    cnt = all_data_.loc[all_data_['store_nbr'] == j, 'transactions'].count()

    all_data.loc[(all_data['transactions'].isna()) & (all_data['year'] == i) & (all_data['store_nbr'] == j), 'transactions'] = sum / cnt


for i in col:
  all_data_ = gb.get_group(i).copy()

  sum = all_data_['transactions'].sum()
  cnt = all_data_['transactions'].count()

  all_data.loc[(all_data['transactions'].isna()) & (all_data['year'] == i), 'transactions'] = sum / cnt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: i

In [101]:
# distinguish holiday and it takes around 1min
list_ = ['type', 'locale', 'locale_name']

for j in list_:
  col = holidays_events[j].unique()
  for i in col:
    all_data['holiday_'+i] = False

  for i in range(0, len(holidays_events)):
    all_data.loc[all_data['date'] == holidays_events['date'][i], 'holiday_'+holidays_events[j][i]] = True

In [102]:
oil['year'] = pd.to_numeric(oil['date'].str[:4])
oil['month'] = pd.to_numeric(oil['date'].str[5:7])
oil['day'] = pd.to_numeric(oil['date'].str[8:10])
oil['date_'] = (oil['year']*10000) + (oil['month']*100) + oil['day']


oil = oil[oil['dcoilwtico'].notna()]

In [103]:
# it takes around half-min

all_data['oil_price'] = 0

for idx, row in oil.iterrows():
  all_data.loc[all_data['date_'] == row['date_'], 'oil_price'] = row['dcoilwtico']

In [104]:
group_flag = 'year'

col = all_data['year'].unique()
col2 = all_data['month'].unique()

gb = all_data.groupby(by=group_flag)


for i in col:
  all_data_ = gb.get_group(i).copy()

  for j in col2:
    sum = all_data_.loc[all_data_['month'] == j, 'oil_price'].sum()
    cnt = all_data_.loc[all_data_['month'] == j, 'oil_price'].count()

    all_data.loc[(all_data['oil_price'] == 0) & (all_data['year'] == i) & (all_data['month'] == j), 'oil_price'] = sum / cnt



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


In [105]:
na_check = all_data.isna().sum() / len(all_data)
na_check

id                                        0.000000
date                                      0.000000
store_nbr                                 0.000000
family                                    0.000000
sales                                     0.009412
onpromotion                               0.000000
year                                      0.000000
month                                     0.000000
day                                       0.000000
week                                      0.000000
date_                                     0.000000
transactions                              0.000000
family_value                              0.000000
family_AUTOMOTIVE                         0.000000
family_BABY CARE                          0.000000
family_BEAUTY                             0.000000
family_BEVERAGES                          0.000000
family_BOOKS                              0.000000
family_BREAD/BAKERY                       0.000000
family_CELEBRATION             

In [106]:
nTrain = len(train)
train = all_data[:nTrain]
test = all_data[nTrain:]

all_data.head()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,week,date_,transactions,family_value,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,family_BREAD/BAKERY,family_CELEBRATION,family_CLEANING,family_DAIRY,family_DELI,family_EGGS,family_FROZEN FOODS,family_GROCERY I,family_GROCERY II,family_HARDWARE,family_HOME AND KITCHEN I,family_HOME AND KITCHEN II,family_HOME APPLIANCES,family_HOME CARE,family_LADIESWEAR,family_LAWN AND GARDEN,family_LINGERIE,"family_LIQUOR,WINE,BEER",family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,holiday_Holiday,holiday_Transfer,holiday_Additional,holiday_Bridge,holiday_Work Day,holiday_Event,holiday_Local,holiday_Regional,holiday_National,holiday_Manta,holiday_Cotopaxi,holiday_Cuenca,holiday_Libertad,holiday_Riobamba,holiday_Puyo,holiday_Guaranda,holiday_Imbabura,holiday_Latacunga,holiday_Machala,holiday_Santo Domingo,holiday_El Carmen,holiday_Cayambe,holiday_Esmeraldas,holiday_Ecuador,holiday_Ambato,holiday_Ibarra,holiday_Quevedo,holiday_Santo Domingo de los Tsachilas,holiday_Santa Elena,holiday_Quito,holiday_Loja,holiday_Salinas,holiday_Guayaquil,oil_price
561,561,2013-01-01,25,AUTOMOTIVE,0.0,0.0,2013,1,1,1,20130101,770.0,5.947114,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,64.19
562,562,2013-01-01,25,BABY CARE,0.0,0.0,2013,1,1,1,20130101,770.0,436.931976,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,64.19
563,563,2013-01-01,25,BEAUTY,2.0,0.0,2013,1,1,1,20130101,770.0,3.660048,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,64.19
564,564,2013-01-01,25,BEVERAGES,810.0,0.0,2013,1,1,1,20130101,770.0,1287.717597,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,64.19
565,565,2013-01-01,25,BOOKS,0.0,0.0,2013,1,1,1,20130101,770.0,436.931976,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,64.19


In [107]:
cormat = train.corr()
cormat = cormat.index[abs(cormat['sales']) > 0]
train[cormat].corr()

,id,store_nbr,sales,onpromotion,year,month,day,week,date_,transactions,family_value,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,family_BREAD/BAKERY,family_CELEBRATION,family_CLEANING,family_DAIRY,family_DELI,family_EGGS,family_FROZEN FOODS,family_GROCERY I,family_GROCERY II,family_HARDWARE,family_HOME AND KITCHEN I,family_HOME AND KITCHEN II,family_HOME APPLIANCES,family_HOME CARE,family_LADIESWEAR,family_LAWN AND GARDEN,family_LINGERIE,"family_LIQUOR,WINE,BEER",family_MAGAZINES,family_MEATS,family_PERSONAL CARE,family_PET SUPPLIES,family_PLAYERS AND ELECTRONICS,family_POULTRY,family_PREPARED FOODS,family_PRODUCE,family_SCHOOL AND OFFICE SUPPLIES,family_SEAFOOD,holiday_Holiday,holiday_Transfer,holiday_Additional,holiday_Bridge,holiday_Work Day,holiday_Event,holiday_Local,holiday_Regional,holiday_National,holiday_Manta,holiday_Cotopaxi,holiday_Cuenca,holiday_Libertad,holiday_Riobamba,holiday_Puyo,holiday_Guaranda,holiday_Imbabura,holiday_Latacunga,holiday_Machala,holiday_Santo Domingo,holiday_El Carmen,holiday_Cayambe,holiday_Esmeraldas,holiday_Ecuador,holiday_Ambato,holiday_Ibarra,holiday_Quevedo,holiday_Santo Domingo de los Tsachilas,holiday_Santa Elena,holiday_Quito,holiday_Loja,holiday_Salinas,holiday_Guayaquil,oil_price
id,1.000000e+00,3.013308e-04,0.085784,0.213851,9.776028e-01,6.743357e-02,5.915116e-03,-1.561462e-03,9.825482e-01,-2.748810e-02,3.467746e-02,-3.265020e-06,-3.060956e-06,-2.856892e-06,-2.652829e-06,-2.448765e-06,-2.244701e-06,-2.040637e-06,-1.836574e-06,-1.632510e-06,-1.428446e-06,-1.224382e-06,-1.020319e-06,-8.162550e-07,-6.121912e-07,-4.081275e-07,-2.040637e-07,3.857899e-18,2.040637e-07,4.081275e-07,6.121912e-07,8.162550e-07,1.020319e-06,1.224382e-06,1.428446e-06,1.632510e-06,1.836574e-06,2.040637e-06,2.244701e-06,2.448765e-06,2.652829e-06,2.856892e-06,3.060956e-06,3.265020e-06,4.591015e-03,5.873133e-02,2.402555e-02,9.457737e-03,-2.987123e-02,5.749770e-02,2.214074e-02,3.658689e-03,4.429614e-02,-5.960775e-03,-2.593105e-03,1.346922e-02,-1.133782e-03,8.728571e-03,2.009376e-03,6.207377e-03,6.948624e-03,6.441933e-03,6.948624e-03,7.846670e-03,7.846670e-03,1.009178e-02,1.155111e-02,4.429614e-02,-2.877939e-03,-2.534459e-03,-1.631087e-03,1.380151e-03,1.480526e-03,6.146709e-03,4.592139e-03,5.997383e-03,1.453172e-02,-7.637043e-01
store_nbr,3.013308e-04,1.000000e+00,0.041196,0.002442,-3.996019e-18,7.311310e-19,-3.096318e-20,1.093411e-20,3.064634e-18,1.799079e-01,-2.438582e-19,-8.308488e-23,-9.693236e-22,8.031538e-22,-8.308488e-23,-8.308488e-23,8.031538e-22,8.031538e-22,-8.308488e-23,-8.308488e-23,-8.308488e-23,-8.308488e-23,-1.190883e-21,-3.046446e-22,-9.693236e-22,-5.262042e-22,-5.262042e-22,-3.046446e-22,-8.308488e-23,-3.046446e-22,-8.308488e-23,-5.262042e-22,1.384748e-22,-5.262042e-22,3.600345e-22,-8.308488e-23,-8.308488e-23,-5.262042e-22,8.031538e-22,3.600345e-22,3.600345e-22,3.877294e-22,-2.769496e-23,4.431194e-22,-1.161386e-19,3.294938e-19,2.923031e-19,-1.895206e-19,5.198255e-20,1.256671e-19,-4.316150e-20,3.289387e-20,2.179940e-19,7.307660e-21,2.964510e-20,-1.458784e-19,5.198255e-20,0.000000e+00,9.665744e-20,-9.348760e-20,5.198255e-20,8.987377e-20,5.198255e-20,-3.736723e-20,-3.736723e-20,5.198255e-20,5.198255e-20,2.179940e-19,3.498772e-20,2.471058e-20,2.471058e-20,7.464376e-20,7.464376e-20,-1.064131e-19,2.471058e-20,-2.522259e-20,-1.794723e-19,1.872290e-18
sales,8.578367e-02,4.119605e-02,1.000000,0.755648,8.109275e-02,1.978967e-02,-1.188702e-02,3.686897e-02,8.185325e-02,2.148432e-01,7.270489e-01,-5.641379e-02,-5.737479e-02,-5.679646e-02,3.253240e-01,-5.738116e-02,1.693347e-02,-5.604977e-02,1.146390e-01,5.636641e-02,-1.486093e-02,-2.989413e-02,-3.256561e-02,5.484896e-01,-5.393012e-02,-5.720999e-02,-5.410877e-02,-5.471000e-02,-5.731913e-02,-2.912775e-02,-5.624385e-02,-5.642434e-02,-5.624040e-02,-4.372713e-02,-5.692265e-02,-2.554731e-03,-1.401115e-02,-5.676349e-02,-5.640006e-02,-1.161958e-03,-4.186915e-02,1.590635e-01,-5.691743e-02,-5.383722e-02,-4.044242e-04,7.315374e-03,1.790395e

In [108]:
all_data.columns

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'year',
       'month', 'day', 'week', 'date_', 'transactions', 'family_value',
       'family_AUTOMOTIVE', 'family_BABY CARE', 'family_BEAUTY',
       'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY',
       'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI',
       'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I',
       'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I',
       'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES',
       'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN',
       'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES',
       'family_MEATS', 'family_PERSONAL CARE', 'family_PET SUPPLIES',
       'family_PLAYERS AND ELECTRONICS', 'family_POULTRY',
       'family_PREPARED FOODS', 'family_PRODUCE',
       'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD',
       'holiday_Holiday', 'holiday_Transfer', 'hol

In [114]:
col_x = ['store_nbr', 'onpromotion', 'year',
       'month', 'day', 'week', 'date_', 'transactions', 'family_value',
       'family_AUTOMOTIVE', 'family_BABY CARE', 'family_BEAUTY',
       'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY',
       'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI',
       'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I',
       'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I',
       'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES',
       'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN',
       'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES',
       'family_MEATS', 'family_PERSONAL CARE', 'family_PET SUPPLIES',
       'family_PLAYERS AND ELECTRONICS', 'family_POULTRY',
       'family_PREPARED FOODS', 'family_PRODUCE',
       'family_SCHOOL AND OFFICE SUPPLIES', 'family_SEAFOOD',
       'holiday_Holiday', 'holiday_Transfer', 'holiday_Additional',
       'holiday_Bridge', 'holiday_Work Day', 'holiday_Event', 'holiday_Local',
       'holiday_Regional', 'holiday_National', 'holiday_Manta',
       'holiday_Cotopaxi', 'holiday_Cuenca', 'holiday_Libertad',
       'holiday_Riobamba', 'holiday_Puyo', 'holiday_Guaranda',
       'holiday_Imbabura', 'holiday_Latacunga', 'holiday_Machala',
       'holiday_Santo Domingo', 'holiday_El Carmen', 'holiday_Cayambe',
       'holiday_Esmeraldas', 'holiday_Ecuador', 'holiday_Ambato',
       'holiday_Ibarra', 'holiday_Quevedo',
       'holiday_Santo Domingo de los Tsachilas', 'holiday_Santa Elena',
       'holiday_Quito', 'holiday_Loja', 'holiday_Salinas', 'holiday_Guayaquil',
       'oil_price']
col_y = 'sales'

col_x

['store_nbr',
 'onpromotion',
 'year',
 'month',
 'day',
 'week',
 'date_',
 'transactions',
 'family_value',
 'family_AUTOMOTIVE',
 'family_BABY CARE',
 'family_BEAUTY',
 'family_BEVERAGES',
 'family_BOOKS',
 'family_BREAD/BAKERY',
 'family_CELEBRATION',
 'family_CLEANING',
 'family_DAIRY',
 'family_DELI',
 'family_EGGS',
 'family_FROZEN FOODS',
 'family_GROCERY I',
 'family_GROCERY II',
 'family_HARDWARE',
 'family_HOME AND KITCHEN I',
 'family_HOME AND KITCHEN II',
 'family_HOME APPLIANCES',
 'family_HOME CARE',
 'family_LADIESWEAR',
 'family_LAWN AND GARDEN',
 'family_LINGERIE',
 'family_LIQUOR,WINE,BEER',
 'family_MAGAZINES',
 'family_MEATS',
 'family_PERSONAL CARE',
 'family_PET SUPPLIES',
 'family_PLAYERS AND ELECTRONICS',
 'family_POULTRY',
 'family_PREPARED FOODS',
 'family_PRODUCE',
 'family_SCHOOL AND OFFICE SUPPLIES',
 'family_SEAFOOD',
 'holiday_Holiday',
 'holiday_Transfer',
 'holiday_Additional',
 'holiday_Bridge',
 'holiday_Work Day',
 'holiday_Event',
 'holiday_Local',

In [61]:
cormat = all_data.corr()

dump = cormat.index[abs(cormat['sales']) >= 0.1]
dump

Index(['sales', 'onpromotion', 'transactions', 'family_value',
       'family_BEVERAGES', 'family_CLEANING', 'family_GROCERY I',
       'family_PRODUCE'],
      dtype='object')

In [115]:
from sklearn.model_selection import train_test_split

train_x = train[col_x]
train_y = np.log1p(train['sales'])

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, shuffle=True)

test_id_idx = test.index

test_x = test[col_x]

In [116]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_x, train_y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [117]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
pred_train = lr.predict(train_x)
pred_val = lr.predict(val_x)

print('train mae score: ', mean_absolute_error(train_y, pred_train))
print('val mae score:', mean_absolute_error(val_y, pred_val))

# train mae score:  0.9710107259396092
# val mae score: 0.9690206192230746
# real score : 1.34725


train mae score:  0.9705034224865617
val mae score: 0.9694707268616135


In [118]:
#xgb.fit(train_x, train_y)
pred = lr.predict(test_x)

In [119]:
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/sample_submission.csv')

In [120]:
#pred = grid_search.predict(test_x)
#submission['sales'] = pred
submission['sales'] = np.expm1(pred)
submission.to_csv('/content/drive/MyDrive/Colab Notebooks/StoreSales/submission.csv', index=False)